In [1]:
import sys
sys.path.insert(0,'..')

In [2]:
from food.psql import *
from mytools.tools import get_logger
logger = get_logger(engine,'bot_logs','food')
logger.debug({'msg':'starting bot'})
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.utils.callback_data import CallbackData
import typing
import numpy as np
import io


from food.paths import *


API_TOKEN = bot_token

from food.paths import *
#from food.search import *
import pandas as  pd
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

import nest_asyncio
nest_asyncio.apply()


dev


In [3]:

from food.search import *
from mytools.psql import *

Since no pretrained weights have been provided, we load the reference pretrained DINO weights.


/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (UnsupervisedMetrics). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


/home/dima/FoodSeg103-Benchmark-v1/checkpoints/SETR_Naive_ReLeM


/home/dima/FoodSeg103-Benchmark-v1/mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


load pre-trained weight from imagenet21k


/home/dima/mytools/mytools/visual.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])


In [4]:
async def  async_insert_on_conflict(*args, **qwargs):
    return insert_on_conflict(*args, **qwargs)

In [5]:
async def add_sender(message):
    logger.debug({'func':'add_sender','id_key':'user_id','id_value':message['from']['id'],'msg':'add_sender'})
    sender = message['from'].to_python()
    sender = pd.DataFrame(sender,index=[0]).drop(columns =['is_bot'])
    await async_insert_on_conflict(sender,'users',unique_cols=['id'],engine = engine)

In [6]:
def plot_nutrition(masks):
    attributes = ['energy','protein','carb','fat']
    nutrition ={}
    for m,a in zip(masks,attributes): nutrition[a] = float(m[m!=0].mean())
    return pd.DataFrame(nutrition,index = ['']).fillna(0).astype(int).to_string()

In [7]:
def image2file_obj(img):
    o = io.BytesIO()
    f = img.format if img.format else 'jpeg'
    img.save(o, format=f)
    return o.getvalue()

In [8]:
async def async_image2file_obj(img):
    return image2file_obj(img)

async def async_search(url):
    return search(url)

async def async_visualize_array(*args,**kwargs):
    return visualize_array(*args,**kwargs)

In [23]:
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

In [24]:
dishes_table = Dishes.__table__

add_dish_cb     = CallbackData('add to food log', 'action')
remove_cb       = CallbackData('remove from food log', 'action')

In [25]:
def attribute_score(cals,values,scores):
    arrays = []
    for n in range(len(values)-1):
        arrays.append(np.stack([np.linspace(values[n],values[n+1]),np.flip(np.linspace(scores[n+1],scores[n]))]))
        formula = np.concatenate(arrays,axis=1)
        c = formula[0][formula[0] <cals][-1]

    return formula[1][formula[0]==c].astype(np.int32)[0]

In [26]:
def get_food_score(cals,protein):
    return attribute_score(cals   ,calore_scores[0],calore_scores[1]),attribute_score(protein,protein_scores[0],protein_scores[1])

In [27]:
protein_scores = ([0,10,20,35],
                  [0,80,100,80])
calore_scores  = ([0,  80, 100,150,200,300,400],
                  [100,100, 90,70 ,60 ,50 ,30])

In [28]:
def get_keyboard(t, unit = None):
    markup = types.InlineKeyboardMarkup()
    if t == 'add to food log' :  
        markup.add(types.InlineKeyboardButton('add to food log', callback_data=add_dish_cb.new(action='add_dish')))
         

    elif t == 'remove from food log':

        btns_text = ('remove from food log',)
        markup.add(types.InlineKeyboardButton('remove from food log', callback_data=remove_cb.new(action='remove')))
        
        
        # markup.add(*(types.InlineKeyboardButton(text, callback_data=remove_cb.new(action=text)) for text in btns_text))

    return markup 

In [29]:
@dp.message_handler(commands=['start'])
async def start_command(message: types.Message):
    global m 
    m = message
    logger.debug({'func':'start_command','id_key':'user_id','id_value':message['from']['id'],'msg':'start_command'})
    await add_sender(message)

    await message.reply(""" Take <b>food pictures</b> to improve your diet.\n 

No  more calorie counting, food weight measurement, manual food logging. <b>A single picture per dish is the only thing you need to do</b> .\n
Your photos are returned back to you colorised as a heat map. Just choose more of coloured in green  next time and less of colorer in  red.\n
Get your <b>nutrition score</b> updated with every meal, try to keep it high.\n
<b>Calorie density</b> is a scientific approach that allows to <b>eat till satisfaction while still cutting back on calories</b>. That idea is implemented <b>with the power of AI</b> to be as easy to follow as taking pictures.\n


<b>how it gain great results:</b>\n
- take photos of all the foods <b>from the same distance</b> and with the same focus distance each time\n
- <b>use flash</b>\n

Now <b>take a picture of your next dish</b> with the bot!""",parse_mode = 'HTML')

In [30]:
# Take <b>pictures of your food</b> to know how healthy it is with the power of <b>AI</b>.\n\
    
#     Get your <b>nutrition score</b> updated every time you take a meal.\n \
#     Keep high food score to stay fit and healthy.\n<b>Take a picture of your next dish with the bot!</b>

In [31]:
@dp.message_handler(content_types=ContentType.PHOTO,state='*')
async def process_photo(message: types.Message, state: FSMContext):
    logger.debug({'func':'process_photo','id_key':'user_id','id_value':message['from']['id'],'msg':'process_photo started'})
    
    processing_reply = await message.reply("""\xa0 your picture is being processed ...""",
                           parse_mode = 'HTML')
    
    await types.ChatActions.upload_photo()
    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    
    # image_url    = await photo.get_url()
    image_url      = f'https://dima.grankin.eu/reference_images/{photo["file_id"]}'
    
    img,clip_df,masks,urls,stats = await async_search(image_url)
    
    print('after search')

    dish = clip_df[['id','score','area']]
    dish['photo_id']         = photo['file_id']
    dish['photo_message_id'] = message['message_id']
    sender = message['from'].to_python()
    dish['user_id'] = sender['id']
    dish['ml_version'] = 0.4 
    dish['timestamp']=pd.Timestamp.utcnow()
    dish = dish.rename(columns = {"id":'food_id'})
    
    output = '; '.join(clip_df['description'].tolist())

    print('downloaded')

    img_o = await async_image2file_obj(img)
    
    await processing_reply.delete()
    reply = await message.reply_photo(img_o,caption=f'<i>per 100 gram</i>:\n{plot_nutrition(masks)} \n\n{output}', reply_markup=get_keyboard('add to food log'),parse_mode = 'HTML')
    dish['message_id'] = reply['message_id']
    
    dish.to_sql('dishes',con=engine,if_exists='append',index = False,schema = 'food')
    
    logger.debug({'func':'process_photo','id_key':'user_id','id_value':message['from']['id'],'msg':'process_photo finished'})
    
    

In [32]:
async def get_today_consumed(user_id):

    today_consumed = pd.read_sql( f"""select f.energy, f.protein, d.area,d.timestamp
                           from food.foods f
                           join food.dishes d on (f.id = d.food_id)
                           where d.user_id = {user_id} and 
                                 d.timestamp > now() - interval '24 hours' and
                                 d.added is true""" ,engine).set_index("timestamp")

    
    area = today_consumed['area'].sum()
    if area>0:
        cals = np.average(today_consumed['energy'],weights=today_consumed['area'])
        prts = np.average(today_consumed['protein'],weights=today_consumed['area'])
        cals_score, prts_score =  get_food_score(cals,prts)
        food_score = int((cals_score*2+prts_score)/3)


        return int(food_score),int(cals),int(prts),area
    
    else: return None,None,None,0

In [33]:
async def add_remove(query,add):
    
    msg = query.to_python()['message']['caption']
    msg = msg.split('\xa0')[0] if '\xa0' in msg else msg
    
    stmt = (
        dishes_table.update()
                    .where(dishes_table.c.message_id == query['message']['message_id'])
                    .values(added=add)
                    .returning(dishes_table.c.id)
        )
    session.execute(stmt)
    session.commit()
    
    user_id = query['from']['id']
    
    food_score,cals,prts,area = await get_today_consumed(user_id)
    
    if add:
        if area>300000: 
            msg = f"{msg}\n\n\xa0your <b>nutrition score</b> for the last 24 hours is <b>{food_score}</b>"
        else:
            msg = f"{msg}\n\n\xa0keep adding your dishes to get your food score"
            
        
    keyboard = 'remove from food log' if add else 'add to food log' 

    await bot.edit_message_caption(query.from_user.id,
                                    query.message.message_id,
                                    caption = msg,
                                    reply_markup=get_keyboard(keyboard),
                                    parse_mode = 'HTML')
    

In [34]:
#add_dish pushed
@dp.callback_query_handler(add_dish_cb.filter(action=['add_dish']))
async def add_dish(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    logger.debug({'func':'add_dish','id_key':'user_id','id_value':query['from']['id'],'msg':'add_dish'})
    
    await add_remove(query,True)

In [35]:
#remove_dish pushed
@dp.callback_query_handler(remove_cb.filter(action=['remove']))
async def remove_dish(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    logger.debug({'func':'remove_dish','id_key':'user_id','id_value':query['from']['id'],'msg':'remove_dish'})
    
    await add_remove(query,False)

In [ ]:
if __name__ == '__main__': executor.start_polling(dp)

Polling is stopped.


In [57]:
message = m